In [19]:
import pandas as pd

# Reading fake news dataset into a pandas dataframe

fake_df = pd.read_csv('datasets/Fake.csv')

fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [20]:
#Reading true news dataset into a pandas dataframe

true_df = pd.read_csv('datasets/True.csv')

true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [21]:
# Checking the info for null values - neither dataset has any

fake_df.info()
true_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB


In [22]:
import nltk 
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\nbohm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [23]:
fake_df['text'] = fake_df['text'].apply(word_tokenize)

fake_df['title'] = fake_df['title'].apply(word_tokenize)

fake_df.head()

,title,text,subject,date
0,"[Donald, Trump, Sends, Out, Embarrassing, New,...","[Donald, Trump, just, couldn, t, wish, all, Am...",News,"December 31, 2017"
1,"[Drunk, Bragging, Trump, Staffer, Started, Rus...","[House, Intelligence, Committee, Chairman, Dev...",News,"December 31, 2017"
2,"[Sheriff, David, Clarke, Becomes, An, Internet...","[On, Friday, ,, it, was, revealed, that, forme...",News,"December 30, 2017"
3,"[Trump, Is, So, Obsessed, He, Even, Has, Obama...","[On, Christmas, day, ,, Donald, Trump, announc...",News,"December 29, 2017"
4,"[Pope, Francis, Just, Called, Out, Donald, Tru...","[Pope, Francis, used, his, annual, Christmas, ...",News,"December 25, 2017"


In [24]:
true_df['text'] = true_df['text'].apply(word_tokenize)

true_df['title'] = true_df['title'].apply(word_tokenize)

true_df.head()

,title,text,subject,date
0,"[As, U.S., budget, fight, looms, ,, Republican...","[WASHINGTON, (, Reuters, ), -, The, head, of, ...",politicsNews,"December 31, 2017"
1,"[U.S., military, to, accept, transgender, recr...","[WASHINGTON, (, Reuters, ), -, Transgender, pe...",politicsNews,"December 29, 2017"
2,"[Senior, U.S., Republican, senator, :, 'Let, M...","[WASHINGTON, (, Reuters, ), -, The, special, c...",politicsNews,"December 31, 2017"
3,"[FBI, Russia, probe, helped, by, Australian, d...","[WASHINGTON, (, Reuters, ), -, Trump, campaign...",politicsNews,"December 30, 2017"
4,"[Trump, wants, Postal, Service, to, charge, 'm...","[SEATTLE/WASHINGTON, (, Reuters, ), -, Preside...",politicsNews,"December 29, 2017"
